In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

pd.set_option('display.max_rows', None)

matplotlib.rcParams.update({'font.size': 15})

In [ ]:
TIME_RETRIEVAL_REE = 0
TIME_RETRIEVAL_TEE = 10
TIME_RETRIEVAL_REE_WASM = 2
TIME_RETRIEVAL_TEE_WASM = 18

In [ ]:
def compute_mean_std(size, time):
    df = pd.DataFrame(columns = ["Size","Time"])
    df["Size"] = size
    df["Time"] = time
    
    result = pd.DataFrame(columns = ["Mean","Std"])
    result["Mean"] = df.groupby("Size")["Time"].median() #df.groupby("Size")["Time"].apply(stats.gmean)
    result["Std"] = df.groupby("Size")["Time"].std() #df.groupby("Size")["Time"].apply(stats.gstd)
    #result["Size"] = df.groupby("Size").groups.keys()
    
    return result

def compute_mean_std_BAK(name, time):
    return pd.DataFrame(
        {
            "Name": [name],
            "Mean": [time.median()],
            "Std": [time.std()]
        }
    )

def formatForLatex(n, digits_round):
    roundedAsString = str(f"{round(n, digits_round):,}")
    if "." in roundedAsString:
        roundedAsString = roundedAsString.rstrip("0")
        roundedAsString = roundedAsString.rstrip(".")
    return f"{roundedAsString}"

In [ ]:
#
## REE data
#

In [ ]:
# Import the dataset
ree = pd.read_csv("../logs/genann/ree.csv", \
        names = ["Size","ImportFile","LoadData","Training","Predict","IsValid","Error"])

# Remove erroneous entries
ree = ree[ree['IsValid'] == 1].drop(columns = ["IsValid", "Error"])
ree

In [ ]:
# Summary of the gathered timespans
ree_spans = pd.DataFrame()

ree_spans["Size"] = ree["Size"]
ree_spans["ImportFile"] = ree["ImportFile"] - 2*TIME_RETRIEVAL_REE_WASM/2
ree_spans["LoadData"] = ree["LoadData"] - 2*TIME_RETRIEVAL_REE_WASM/2
ree_spans["Training"] = ree["Training"] - 2*TIME_RETRIEVAL_REE_WASM/2
ree_spans["Predict"] = ree["Predict"] - 2*TIME_RETRIEVAL_REE_WASM/2

ree_spans

In [ ]:
# Compute the mean and std
def microsec_to_sec(df):
    return df / 1000 / 1000

ree_exports = {}
ree_exports["ImportFile"] = compute_mean_std(ree_spans["Size"], ree_spans["ImportFile"])
ree_exports["LoadData"] = compute_mean_std(ree_spans["Size"], ree_spans["LoadData"])
ree_exports["Training"] = microsec_to_sec(compute_mean_std(ree_spans["Size"], ree_spans["Training"]))
ree_exports["Predict"] = compute_mean_std(ree_spans["Size"], ree_spans["Predict"])
ree_exports

In [ ]:
#
## TEE data
#

In [ ]:
# Import the datasets
# Import the dataset
tee_columns = ["Size","CollectQuote","NetHandshake","SendQuote","ReceiveData","LoadData", \
               "Training","Predict","IsValid","Error"]
tee = pd.read_csv("../logs/genann/tee.csv", names = tee_columns)

# Remove erroneous entries
tee = tee[tee['IsValid'] == 1].drop(columns = ["IsValid", "Error"])
tee

In [ ]:
# Summary of the gathered timespans
tee_spans = pd.DataFrame()

tee_spans["Size"] = tee["Size"]
tee_spans["CollectQuote"] = tee["CollectQuote"] - 2*TIME_RETRIEVAL_TEE_WASM/2
tee_spans["NetHandshake"] = tee["NetHandshake"] - 2*TIME_RETRIEVAL_TEE_WASM/2
tee_spans["SendQuote"] = tee["SendQuote"] - 2*TIME_RETRIEVAL_TEE_WASM/2
tee_spans["ReceiveData"] = tee["ReceiveData"] - 2*TIME_RETRIEVAL_TEE_WASM/2
tee_spans["LoadData"] = tee["LoadData"] - 2*TIME_RETRIEVAL_TEE_WASM/2
tee_spans["Training"] = tee["Training"] - 2*TIME_RETRIEVAL_TEE_WASM/2
tee_spans["Predict"] = tee["Predict"] - 2*TIME_RETRIEVAL_TEE_WASM/2

tee_spans

In [ ]:
# Compute the mean and std
tee_exports = {}
tee_exports["CollectQuote"] = compute_mean_std(tee_spans["Size"], tee_spans["CollectQuote"])
tee_exports["NetHandshake"] = compute_mean_std(tee_spans["Size"], tee_spans["NetHandshake"])
tee_exports["SendQuote"] = compute_mean_std(tee_spans["Size"], tee_spans["SendQuote"])
tee_exports["ReceiveData"] = compute_mean_std(tee_spans["Size"], tee_spans["ReceiveData"])
tee_exports["LoadData"] = compute_mean_std(tee_spans["Size"], tee_spans["LoadData"])
tee_exports["Training"] = microsec_to_sec(compute_mean_std(tee_spans["Size"], tee_spans["Training"]))
tee_exports["Predict"] = compute_mean_std(tee_spans["Size"], tee_spans["Predict"])
tee_exports

In [ ]:
# Draw the comparison of REE/TEE execution time plot

ree_bar = ree_exports["Training"]["Mean"]
tee_bar = tee_exports["Training"]["Mean"]
df = pd.DataFrame({'REE': ree_bar,
                   'TEE': tee_bar}, index=ree_exports["Training"].index)
ax = df.plot.bar(rot=0) # .line()

In [ ]:
df = pd.DataFrame(
    {
        'CollectQuote': tee_exports["CollectQuote"]["Mean"],
        'NetHandshake': tee_exports["NetHandshake"]["Mean"],
        'SendQuote': tee_exports["SendQuote"]["Mean"],
        'ReceiveData': tee_exports["ReceiveData"]["Mean"]
    },
    index=ree_exports["Training"].index)
ax = df.plot.bar(rot=0, stacked=True)

In [ ]:
df = pd.DataFrame(
    {
        'ReceiveData': tee_exports["ReceiveData"]["Mean"]
    },
    index=ree_exports["Training"].index)
ax = df.plot.bar(rot=0)

In [ ]:
# Export CSV
#
# File format: size, mean, stddev

for key, value in ree_exports.items():
    value.to_csv(f"../logs/genann/ree_{key}.csv")

for key, value in tee_exports.items():
    value.to_csv(f"../logs/genann/tee_{key}.csv")

In [ ]:
# Compute values for the paper

tee_collect_quote_mean = tee_exports["CollectQuote"]["Mean"].mean()
tee_collect_quote_std = tee_exports["CollectQuote"]["Std"].mean()
tee_net_handshake_mean = tee_exports["NetHandshake"]["Mean"].mean()
tee_net_handshake_std = tee_exports["NetHandshake"]["Std"].mean()
tee_send_quote_mean = tee_exports["SendQuote"]["Mean"].mean()
tee_send_quote_std = tee_exports["SendQuote"]["Std"].mean()

tee_receive_data_min_mean = tee_exports["ReceiveData"].loc[tee_exports["ReceiveData"].index == 100]["Mean"].values[0]
tee_receive_data_min_std = tee_exports["ReceiveData"].loc[tee_exports["ReceiveData"].index == 100]["Std"].values[0]
tee_receive_data_max_mean = tee_exports["ReceiveData"].loc[tee_exports["ReceiveData"].index == 1000]["Mean"].values[0]
tee_receive_data_max_std = tee_exports["ReceiveData"].loc[tee_exports["ReceiveData"].index == 1000]["Std"].values[0]


tee_sum_of_wasi_ra_min_in_microsec = round(tee_collect_quote_mean,0) + round(tee_net_handshake_mean,0) \
                                          + round(tee_send_quote_mean,0) + round(tee_receive_data_min_mean,0)
tee_sum_of_wasi_ra_max_in_microsec = round(tee_collect_quote_mean,0) + round(tee_net_handshake_mean,0) \
                                          + round(tee_send_quote_mean,0) + round(tee_receive_data_max_mean,0)

tee_sum_of_wasi_ra_without_data_in_microsec = tee_collect_quote_mean + tee_net_handshake_mean \
                                          + tee_send_quote_mean

training_slowdown = (tee_exports["Training"]["Mean"] / ree_exports["Training"]["Mean"]).median()

In [ ]:
prefix = "Genann"

def write(file, name, number, digits_round = 0):
    f.write(f"\\def\\bm{prefix}{name}{{{formatForLatex(number, digits_round)}}}\n")
        
f = open("../logs/genann/export.tex", "w")
write(f, "CollectQuoteMean", tee_collect_quote_mean / 1000)
#write(f, "CollectQuoteStd", tee_collect_quote_std)
write(f, "NetHandshakeMean", tee_net_handshake_mean / 1000 ** 2, 2)
#write(f, "NetHandshakeStd", tee_net_handshake_std)
write(f, "SendQuoteMean", tee_send_quote_mean / 1000)
#write(f, "SendQuoteStd", tee_send_quote_std)
write(f, "SumOfWasiRaMin", tee_sum_of_wasi_ra_min_in_microsec / 1000 ** 2, 2)
#write(f, "SumOfWasiRaMaxInMicrosec", tee_sum_of_wasi_ra_max_in_microsec)
write(f, "SumOfWasiRaMax", tee_sum_of_wasi_ra_max_in_microsec / 1000 ** 2, 2)
#write(f, "SumOfWasiRaWithoutDataInMicrosec", tee_sum_of_wasi_ra_without_data_in_microsec)
write(f, "SumOfWasiRaWithoutData", tee_sum_of_wasi_ra_without_data_in_microsec / 1000 ** 2, 2)
write(f, "ReceiveDataMinMeanInMicrosec", tee_receive_data_min_mean)
write(f, "ReceiveDataMinStdInMicrosec", tee_receive_data_min_std)
write(f, "ReceiveDataMaxMeanInMicrosec", tee_receive_data_max_mean)
write(f, "ReceiveDataMaxStdInMicrosec", tee_receive_data_max_std)
write(f, "ReceiveDataMinMeanInMs", tee_receive_data_min_mean / 1000, 0)
write(f, "ReceiveDataMaxMeanInMs", tee_receive_data_max_mean / 1000, 0)
write(f, "TrainingSlowdown", training_slowdown, 3)
write(f, "TrainingSpeedUpPercent", (1 - training_slowdown) * 100, 1)
f.close()